In [1]:
import pandas as pd

In [26]:
import mysql.connector as db

In [27]:
import os

In [28]:
customers = pd.read_csv("J:\\Customer Behavior Analysis\\customers.csv")
customer_journey = pd.read_csv("J:\\Customer Behavior Analysis\\customer_journey.csv")
customer_reviews = pd.read_csv("J:\\Customer Behavior Analysis\\customer_reviews.csv")
engagement_data = pd.read_csv("J:\\Customer Behavior Analysis\\engagement_data.csv")
geography = pd.read_csv("J:\\Customer Behavior Analysis\\geography.csv")
products = pd.read_csv("J:\\Customer Behavior Analysis\\products.csv")

In [29]:
# Check for missing values in each DataFrame
print(customer_journey.isnull().sum())
print(customer_reviews.isnull().sum())
print(customers.isnull().sum())
print(engagement_data.isnull().sum())
print(geography.isnull().sum())
print(products.isnull().sum())

JourneyID      0
CustomerID     0
ProductID      0
VisitDate      0
Stage          0
Action         0
Duration      14
dtype: int64
ReviewID      0
CustomerID    0
ProductID     0
ReviewDate    0
Rating        0
ReviewText    0
dtype: int64
CustomerID      0
CustomerName    0
Email           0
Gender          0
Age             0
GeographyID     0
dtype: int64
EngagementID           0
ContentID              0
ContentType            0
Likes                  0
EngagementDate         0
CampaignID             0
ProductID              0
ViewsClicksCombined    0
dtype: int64
GeographyID    0
Country        0
City           0
dtype: int64
ProductID      0
ProductName    0
Category       0
Price          0
dtype: int64


In [30]:
customer_journey['Duration'].fillna(0, inplace=True)

C:\Users\jagad\AppData\Local\Temp\ipykernel_6632\2335063406.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  customer_journey['Duration'].fillna(0, inplace=True)


In [31]:
customer_journey.columns = customer_journey.columns.str.lower()
customer_reviews.columns = customer_reviews.columns.str.lower()
customers.columns = customers.columns.str.lower()
engagement_data.columns = engagement_data.columns.str.lower()
geography.columns = geography.columns.str.lower()
products.columns = products.columns.str.lower()

In [32]:
customer_journey['visitdate'] = pd.to_datetime(customer_journey['visitdate'])
customer_reviews['reviewdate'] = pd.to_datetime(customer_reviews['reviewdate'])
engagement_data['engagementdate'] = pd.to_datetime(engagement_data['engagementdate'])

In [33]:
print(customer_journey.info())
print(customer_reviews.info())
print(customers.info())
print(engagement_data.info())
print(geography.info())
print(products.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   journeyid   100 non-null    int64         
 1   customerid  100 non-null    int64         
 2   productid   100 non-null    int64         
 3   visitdate   100 non-null    datetime64[ns]
 4   stage       100 non-null    object        
 5   action      100 non-null    object        
 6   duration    100 non-null    float64       
dtypes: datetime64[ns](1), float64(1), int64(3), object(2)
memory usage: 5.6+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   reviewid    100 non-null    int64         
 1   customerid  100 non-null    int64         
 2   productid   100 non-null    int64         
 3   reviewdate  100 non-null    dat

In [34]:
user = 'root'
password = 'Jagadish@1430'
host = 'localhost'

In [35]:
db_connection = db.connect(
    host=host,
    user=user,
    password=password
)
db_connection
curr = db_connection.cursor()


In [50]:
curr.execute('create database if not exists ShopEasy')
curr.execute("USE ShopEasy")

In [51]:
# Function to insert DataFrame into MySQL table
def insert_dataframe_to_mysql(table_name, dataframe):
    # Create the table if it doesn't exist
    create_table_query = f"""
    CREATE TABLE IF NOT EXISTS {table_name} (
        {', '.join([f'{col} VARCHAR(255)' for col in dataframe.columns])}
    )
    """
    curr.execute(create_table_query)
# Insert data into the table
    for _, row in dataframe.iterrows():
        insert_query = f"""
        INSERT INTO {table_name} ({', '.join(dataframe.columns)})
        VALUES ({', '.join(['%s'] * len(row))})
        """
        curr.execute(insert_query, tuple(row.astype(str)))  # Convert all values to strings
    db_connection.commit()


In [52]:
# Insert each DataFrame into MySQL
insert_dataframe_to_mysql('customer_journey', customer_journey)
insert_dataframe_to_mysql('customer_reviews', customer_reviews)
insert_dataframe_to_mysql('customers', customers)
insert_dataframe_to_mysql('engagement_data', engagement_data)
insert_dataframe_to_mysql('geography', geography)
insert_dataframe_to_mysql('products', products)

# Close the connection
curr.close()
db_connection.close()

print("Data has been successfully inserted into the 'shopeasy' database!")

Data has been successfully inserted into the 'shopeasy' database!
